In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
import os
import numpy as np
np.random.seed(777)
import math
import keras
import keras.backend as K
import h5py
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam, SGD, RMSprop
from keras.models import Sequential
from keras.models import Model
from keras.layers import Input, Activation, merge, Dense, Flatten, Dropout, concatenate
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import BatchNormalization, add, GlobalAveragePooling2D
from keras.utils.np_utils import to_categorical
from keras.optimizers import Adam
from sklearn.metrics import accuracy_score,roc_curve, confusion_matrix, roc_auc_score, auc, f1_score
from keras.regularizers import l2
from keras.applications.xception import Xception, preprocess_input
from keras.applications.mobilenet import MobileNet
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications import DenseNet201
from keras.applications import DenseNet121
from keras.regularizers import l2, l1

from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, Lambda,Concatenate
from keras.layers import SeparableConv2D, AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D, Add

import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams["axes.grid"] = False
plt.rcParams.update({'font.size': 20})

Using TensorFlow backend.


In [3]:
train_dir = 'data/train/'
test_dir = "data/test/"

# os.mkdir("extracted_features")
extracted_features_dir = "extracted_features/"
model_name = "Dense201_Xception_Inceptionv3"

In [4]:
import keras
import tensorflow as tf
import keras.backend as K

print("Keras Version", keras.__version__)
print("tensorflow Version", tf.__version__)
print("dim_ordering:", K.image_dim_ordering())

Keras Version 2.2.4
tensorflow Version 1.10.0
dim_ordering: tf


In [5]:
batch_size = 512
img_height, img_width = 128, 128
input_shape = (img_height, img_width, 3)
epochs = 1000

In [6]:
# for root,dirs,files in os.walk(train_dir):
#     print (root, len(files))

# print("*"*30)
# for root,dirs,files in os.walk(test_dir):
#     print (root, len(files))

In [7]:
def preprocess_input(x):
    # 'RGB'->'BGR'
    x = x[:, :, ::-1]
    # Zero-center by imagenet mean pixel
    x[:, :, 0] -= 103.939
    x[:, :, 1] -= 116.779
    x[:, :, 2] -= 123.68
    return x

In [8]:
random_seed = np.random.seed(1142)

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
#     featurewise_center=True,
#     featurewise_std_normalization=True,
    preprocessing_function = preprocess_input,
    validation_split= 0.2,
    zoom_range=0.2,
    shear_range=0.2)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'training',
    class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'validation',
    class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1. / 255, preprocessing_function = preprocess_input)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    class_mode='categorical')

Found 319152 images belonging to 6 classes.
Found 79784 images belonging to 6 classes.
Found 13578 images belonging to 6 classes.


In [9]:
nb_train_samples = len(train_generator.filenames)
nb_validation_samples = len(validation_generator.filenames)
nb_test_samples = len(test_generator.filenames)

predict_size_train = int(math.ceil(nb_train_samples / batch_size))
predict_size_validation = int(math.ceil(nb_validation_samples / batch_size))
predict_size_test = int(math.ceil(nb_test_samples / batch_size))

num_classes = len(train_generator.class_indices)

print("nb_train_samples:", nb_train_samples)
print("nb_validation_samples:", nb_validation_samples)
print("nb_test_samples:", nb_test_samples)

print("\npredict_size_train:", predict_size_train)
print("predict_size_validation:", predict_size_validation)
print("predict_size_test:", predict_size_test)

print("\n num_classes:", num_classes)

nb_train_samples: 319152
nb_validation_samples: 79784
nb_test_samples: 13578

predict_size_train: 624
predict_size_validation: 156
predict_size_test: 27

 num_classes: 6


In [10]:
# from keras.backend.tensorflow_backend import get_session
# from keras.backend.tensorflow_backend import clear_session
# from keras.backend.tensorflow_backend import set_session

# def reset_keras_tf_session():
#     """
#     this function clears the gpu memory and set the 
#     tf session to not use the whole gpu
#     """
#     sess = get_session()
#     clear_session()
#     sess.close()
#     sess = get_session()

# #     config = tf.ConfigProto()
# #     config.gpu_options.allow_growth = True
# #     set_session(tf.Session(config=config))


# reset_keras_tf_session()

In [ ]:
from keras.applications.xception import Xception, preprocess_input
from keras.applications import DenseNet201
from keras.applications.inception_v3 import InceptionV3


input_tensor = Input(shape = input_shape)  

base_model1=DenseNet201(input_shape= input_shape,weights='imagenet', include_top=False, input_tensor=input_tensor)
base_model2=Xception(input_shape= input_shape,weights='imagenet', include_top=False, input_tensor=input_tensor)
base_model3=InceptionV3(input_shape= input_shape,weights='imagenet', include_top=False, input_tensor=input_tensor)

x1 = base_model1.output
x1 = GlobalAveragePooling2D()(x1)

x2 = base_model2.output
x2 = GlobalAveragePooling2D()(x2)

x3 = base_model3.output
x3 = GlobalAveragePooling2D()(x3)

merge = concatenate([x1, x2, x3])
predictions = Dense(num_classes, activation='softmax')(merge)

model = Model(inputs=input_tensor,outputs=predictions)

In [ ]:
for i, layer in enumerate(model.layers):
    print(i, layer.name)

In [ ]:
c1 = model.layers[9].output 
c1 = GlobalAveragePooling2D()(c1)       

c2 = model.layers[55].output
c2 = GlobalAveragePooling2D()(c2)       

c3 = model.layers[143].output
c3 = GlobalAveragePooling2D()(c3)       

c4 = model.layers[483].output
c4 = GlobalAveragePooling2D()(c4) 

c5 = model.layers[572].output
c5 = GlobalAveragePooling2D()(c5) 

c6 = model.layers[586].output
c6 = GlobalAveragePooling2D()(c6) 

c7 = model.layers[602].output
c7 = GlobalAveragePooling2D()(c7) 

c8 = model.layers[636].output
c8 = GlobalAveragePooling2D()(c8) 

c9 = model.layers[679].output
c9 = GlobalAveragePooling2D()(c9) 

c10 = model.layers[702].output
c10 = GlobalAveragePooling2D()(c10) 

c11 = model.layers[722].output
c11 = GlobalAveragePooling2D()(c11) 

c12 = model.layers[738].output
c12 = GlobalAveragePooling2D()(c12) 

c13 = model.layers[749].output
c13 = GlobalAveragePooling2D()(c13) 

c14 = model.layers[783].output
c14 = GlobalAveragePooling2D()(c14) 

c15 = model.layers[847].output
c15 = GlobalAveragePooling2D()(c15) 

c16 = model.layers[911].output
c16 = GlobalAveragePooling2D()(c16) 

c17 = model.layers[975].output
c17 = GlobalAveragePooling2D()(c17) 

c18 = model.layers[1042].output
c18 = GlobalAveragePooling2D()(c18) 

c19 = model.layers[1070].output
c19 = GlobalAveragePooling2D()(c19) 

c20 = model.layers[1106].output
c20 = GlobalAveragePooling2D()(c20) 

c21 = model.layers[1125].output
c21 = GlobalAveragePooling2D()(c21) 

con = concatenate([c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12, c13, c14, c15, c16, c17, c18, c19, c20, c21])

bottleneck_final_model = Model(inputs=model.input, outputs=con)

In [ ]:
bottleneck_final_model.summary()

In [ ]:
bottleneck_features_train = bottleneck_final_model.predict_generator(train_generator, predict_size_train)
np.save(extracted_features_dir+'bottleneck_features_train_'+model_name+'.npy', bottleneck_features_train)

In [ ]:
bottleneck_features_validation = bottleneck_final_model.predict_generator(validation_generator, predict_size_validation)
np.save(extracted_features_dir+'bottleneck_features_validation_'+model_name+'.npy', bottleneck_features_validation)

bottleneck_features_test = bottleneck_final_model.predict_generator(test_generator, predict_size_test)
np.save(extracted_features_dir+'bottleneck_features_test_'+model_name+'.npy', bottleneck_features_test)

In [11]:
train_data = np.load(extracted_features_dir+'bottleneck_features_train_'+model_name+'.npy')
validation_data = np.load(extracted_features_dir+'bottleneck_features_validation_'+model_name+'.npy')
test_data = np.load(extracted_features_dir+'bottleneck_features_test_'+model_name+'.npy')

train_labels = train_generator.classes
train_labels = to_categorical(train_labels, num_classes=num_classes)

validation_labels = validation_generator.classes
validation_labels = to_categorical(validation_labels, num_classes=num_classes)

test_labels = test_generator.classes
test_labels = to_categorical(test_labels, num_classes=num_classes)

In [12]:
# dropout_rate = 0.5
model = Sequential()
# model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(256, activation="relu", kernel_regularizer=l2(1e-06), bias_regularizer=l2(0.1), activity_regularizer=l1(1e-07)))

model.add(Dense(512, activation="relu", kernel_regularizer=l2(1e-06), bias_regularizer=l2(0.1), activity_regularizer=l1(1e-07)))

model.add(Dense(32, activation="relu", kernel_regularizer=l2(1e-06), bias_regularizer=l2(0.1), activity_regularizer=l1(1e-07)))

model.add(Dense(256, activation="relu", kernel_regularizer=l2(1e-06), bias_regularizer=l2(0.1), activity_regularizer=l1(1e-07)))

model.add(Dense(64, activation="relu", kernel_regularizer=l2(1e-06), bias_regularizer=l2(0.1), activity_regularizer=l1(1e-07)))


model.add(Dropout(0.5))


    
model.add(Dense(num_classes, activation="softmax"))

adam_opt=Adam(lr = 1e-05, beta_1=0.8, beta_2=0.3)

model.compile(optimizer=adam_opt, loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_data, train_labels,
                    epochs=1000,
                    batch_size=batch_size,
                    validation_data=(validation_data, validation_labels),
                    verbose= 2)

with open(extracted_features_dir+'history_'+model_name+'.txt','w') as f:
    f.write(str(history.history))

    
    
top_model_path = os.path.join(extracted_features_dir, 'model_'+model_name+'_model.h5')
top_model_weights_path = os.path.join(extracted_features_dir, 'model_'+model_name+'_weights.h5')    
    
model.save_weights(top_model_weights_path)
model.save(top_model_path)

Train on 319152 samples, validate on 79784 samples
Epoch 1/1000
 - 10s - loss: 1.2645 - acc: 0.5241 - val_loss: 0.7945 - val_acc: 0.7571
Epoch 2/1000
 - 8s - loss: 0.7235 - acc: 0.7666 - val_loss: 0.5473 - val_acc: 0.8207
Epoch 3/1000
 - 7s - loss: 0.5708 - acc: 0.8210 - val_loss: 0.4615 - val_acc: 0.8557
Epoch 4/1000
 - 7s - loss: 0.5004 - acc: 0.8464 - val_loss: 0.4163 - val_acc: 0.8695
Epoch 5/1000
 - 7s - loss: 0.4572 - acc: 0.8610 - val_loss: 0.3826 - val_acc: 0.8778
Epoch 6/1000
 - 7s - loss: 0.4277 - acc: 0.8705 - val_loss: 0.3613 - val_acc: 0.8848
Epoch 7/1000
 - 7s - loss: 0.4047 - acc: 0.8775 - val_loss: 0.3472 - val_acc: 0.8891
Epoch 8/1000
 - 7s - loss: 0.3865 - acc: 0.8824 - val_loss: 0.3292 - val_acc: 0.8944
Epoch 9/1000
 - 7s - loss: 0.3729 - acc: 0.8870 - val_loss: 0.3208 - val_acc: 0.8973
Epoch 10/1000
 - 7s - loss: 0.3588 - acc: 0.8911 - val_loss: 0.3153 - val_acc: 0.8974
Epoch 11/1000
 - 7s - loss: 0.3475 - acc: 0.8944 - val_loss: 0.3057 - val_acc: 0.9005
Epoch 12/10

Epoch 96/1000
 - 7s - loss: 0.1272 - acc: 0.9621 - val_loss: 0.1357 - val_acc: 0.9561
Epoch 97/1000
 - 7s - loss: 0.1263 - acc: 0.9625 - val_loss: 0.1300 - val_acc: 0.9581
Epoch 98/1000
 - 8s - loss: 0.1238 - acc: 0.9633 - val_loss: 0.1189 - val_acc: 0.9619
Epoch 99/1000
 - 7s - loss: 0.1222 - acc: 0.9641 - val_loss: 0.1209 - val_acc: 0.9608
Epoch 100/1000
 - 7s - loss: 0.1221 - acc: 0.9643 - val_loss: 0.1210 - val_acc: 0.9616
Epoch 101/1000
 - 7s - loss: 0.1209 - acc: 0.9644 - val_loss: 0.1201 - val_acc: 0.9617
Epoch 102/1000
 - 7s - loss: 0.1199 - acc: 0.9645 - val_loss: 0.1195 - val_acc: 0.9614
Epoch 103/1000
 - 7s - loss: 0.1181 - acc: 0.9655 - val_loss: 0.1186 - val_acc: 0.9616
Epoch 104/1000
 - 7s - loss: 0.1183 - acc: 0.9654 - val_loss: 0.1204 - val_acc: 0.9613
Epoch 105/1000
 - 7s - loss: 0.1167 - acc: 0.9659 - val_loss: 0.1223 - val_acc: 0.9608
Epoch 106/1000
 - 7s - loss: 0.1156 - acc: 0.9661 - val_loss: 0.1193 - val_acc: 0.9619
Epoch 107/1000
 - 7s - loss: 0.1163 - acc: 0.96

Epoch 191/1000
 - 7s - loss: 0.0641 - acc: 0.9828 - val_loss: 0.0980 - val_acc: 0.9701
Epoch 192/1000
 - 7s - loss: 0.0636 - acc: 0.9829 - val_loss: 0.1062 - val_acc: 0.9682
Epoch 193/1000
 - 7s - loss: 0.0635 - acc: 0.9828 - val_loss: 0.0860 - val_acc: 0.9746
Epoch 194/1000
 - 7s - loss: 0.0634 - acc: 0.9826 - val_loss: 0.0945 - val_acc: 0.9721
Epoch 195/1000
 - 7s - loss: 0.0638 - acc: 0.9827 - val_loss: 0.1001 - val_acc: 0.9693
Epoch 196/1000
 - 7s - loss: 0.0628 - acc: 0.9829 - val_loss: 0.0868 - val_acc: 0.9749
Epoch 197/1000
 - 7s - loss: 0.0615 - acc: 0.9834 - val_loss: 0.0917 - val_acc: 0.9733
Epoch 198/1000
 - 7s - loss: 0.0621 - acc: 0.9831 - val_loss: 0.0951 - val_acc: 0.9718
Epoch 199/1000
 - 7s - loss: 0.0625 - acc: 0.9830 - val_loss: 0.0948 - val_acc: 0.9719
Epoch 200/1000
 - 7s - loss: 0.0632 - acc: 0.9827 - val_loss: 0.0916 - val_acc: 0.9724
Epoch 201/1000
 - 7s - loss: 0.0595 - acc: 0.9844 - val_loss: 0.0908 - val_acc: 0.9732
Epoch 202/1000
 - 7s - loss: 0.0604 - acc: 

 - 7s - loss: 0.0386 - acc: 0.9903 - val_loss: 0.0911 - val_acc: 0.9774
Epoch 286/1000
 - 7s - loss: 0.0396 - acc: 0.9900 - val_loss: 0.1008 - val_acc: 0.9744
Epoch 287/1000
 - 7s - loss: 0.0381 - acc: 0.9903 - val_loss: 0.0894 - val_acc: 0.9776
Epoch 288/1000
 - 7s - loss: 0.0401 - acc: 0.9898 - val_loss: 0.0848 - val_acc: 0.9787
Epoch 289/1000
 - 7s - loss: 0.0398 - acc: 0.9899 - val_loss: 0.0927 - val_acc: 0.9763
Epoch 290/1000
 - 7s - loss: 0.0384 - acc: 0.9903 - val_loss: 0.0910 - val_acc: 0.9767
Epoch 291/1000
 - 7s - loss: 0.0405 - acc: 0.9899 - val_loss: 0.1006 - val_acc: 0.9752
Epoch 292/1000
 - 7s - loss: 0.0390 - acc: 0.9900 - val_loss: 0.0807 - val_acc: 0.9795
Epoch 293/1000
 - 7s - loss: 0.0381 - acc: 0.9905 - val_loss: 0.1068 - val_acc: 0.9743
Epoch 294/1000
 - 7s - loss: 0.0393 - acc: 0.9901 - val_loss: 0.1154 - val_acc: 0.9692
Epoch 295/1000
 - 7s - loss: 0.0396 - acc: 0.9901 - val_loss: 0.0857 - val_acc: 0.9782
Epoch 296/1000
 - 7s - loss: 0.0384 - acc: 0.9903 - val_lo

Epoch 380/1000
 - 7s - loss: 0.0314 - acc: 0.9925 - val_loss: 0.0875 - val_acc: 0.9808
Epoch 381/1000
 - 7s - loss: 0.0317 - acc: 0.9924 - val_loss: 0.0896 - val_acc: 0.9798
Epoch 382/1000
 - 7s - loss: 0.0302 - acc: 0.9928 - val_loss: 0.0779 - val_acc: 0.9820
Epoch 383/1000
 - 7s - loss: 0.0312 - acc: 0.9923 - val_loss: 0.0941 - val_acc: 0.9783
Epoch 384/1000
 - 7s - loss: 0.0304 - acc: 0.9928 - val_loss: 0.0898 - val_acc: 0.9795
Epoch 385/1000
 - 7s - loss: 0.0295 - acc: 0.9930 - val_loss: 0.1048 - val_acc: 0.9764
Epoch 386/1000
 - 7s - loss: 0.0280 - acc: 0.9934 - val_loss: 0.0992 - val_acc: 0.9772
Epoch 387/1000
 - 7s - loss: 0.0305 - acc: 0.9929 - val_loss: 0.1243 - val_acc: 0.9710
Epoch 388/1000
 - 7s - loss: 0.0295 - acc: 0.9931 - val_loss: 0.0832 - val_acc: 0.9810
Epoch 389/1000
 - 7s - loss: 0.0319 - acc: 0.9922 - val_loss: 0.0845 - val_acc: 0.9810
Epoch 390/1000
 - 7s - loss: 0.0309 - acc: 0.9928 - val_loss: 0.0845 - val_acc: 0.9808
Epoch 391/1000
 - 7s - loss: 0.0297 - acc: 

...

Epoch 569/1000
 - 7s - loss: 0.0242 - acc: 0.9945 - val_loss: 0.0986 - val_acc: 0.9811
Epoch 570/1000
 - 7s - loss: 0.0223 - acc: 0.9951 - val_loss: 0.1104 - val_acc: 0.9776
Epoch 571/1000
 - 7s - loss: 0.0211 - acc: 0.9955 - val_loss: 0.1053 - val_acc: 0.9796
Epoch 572/1000
 - 7s - loss: 0.0211 - acc: 0.9955 - val_loss: 0.0950 - val_acc: 0.9804
Epoch 573/1000
 - 7s - loss: 0.0220 - acc: 0.9953 - val_loss: 0.1058 - val_acc: 0.9799
Epoch 574/1000
 - 7s - loss: 0.0222 - acc: 0.9952 - val_loss: 0.1214 - val_acc: 0.9772
Epoch 575/1000
 - 7s - loss: 0.0218 - acc: 0.9951 - val_loss: 0.1033 - val_acc: 0.9798
Epoch 576/1000
 - 7s - loss: 0.0255 - acc: 0.9943 - val_loss: 0.0990 - val_acc: 0.9807
Epoch 577/1000
 - 7s - loss: 0.0224 - acc: 0.9950 - val_loss: 0.0976 - val_acc: 0.9809
Epoch 578/1000
 - 7s - loss: 0.0215 - acc: 0.9954 - val_loss: 0.1102 - val_acc: 0.9790
Epoch 579/1000
 - 7s - loss: 0.0199 - acc: 0.9958 - val_loss: 0.0991 - val_acc: 0.9814
Epoch 580/1000
 - 8s - loss: 0.0227 - acc: 

 - 8s - loss: 0.0214 - acc: 0.9955 - val_loss: 0.1019 - val_acc: 0.9810
Epoch 664/1000
 - 7s - loss: 0.0203 - acc: 0.9959 - val_loss: 0.0978 - val_acc: 0.9817
Epoch 665/1000
 - 8s - loss: 0.0198 - acc: 0.9959 - val_loss: 0.1051 - val_acc: 0.9810
Epoch 666/1000
 - 8s - loss: 0.0194 - acc: 0.9960 - val_loss: 0.1001 - val_acc: 0.9816
Epoch 667/1000
 - 8s - loss: 0.0209 - acc: 0.9955 - val_loss: 0.1258 - val_acc: 0.9761
Epoch 668/1000
 - 8s - loss: 0.0216 - acc: 0.9953 - val_loss: 0.0950 - val_acc: 0.9813
Epoch 669/1000
 - 8s - loss: 0.0205 - acc: 0.9956 - val_loss: 0.0964 - val_acc: 0.9822
Epoch 670/1000
 - 8s - loss: 0.0206 - acc: 0.9957 - val_loss: 0.1002 - val_acc: 0.9810
Epoch 671/1000
 - 8s - loss: 0.0206 - acc: 0.9958 - val_loss: 0.0982 - val_acc: 0.9822
Epoch 672/1000
 - 8s - loss: 0.0203 - acc: 0.9958 - val_loss: 0.1124 - val_acc: 0.9782
Epoch 673/1000
 - 8s - loss: 0.0203 - acc: 0.9959 - val_loss: 0.0946 - val_acc: 0.9814
Epoch 674/1000
 - 8s - loss: 0.0194 - acc: 0.9961 - val_lo

Epoch 758/1000
 - 8s - loss: 0.0192 - acc: 0.9960 - val_loss: 0.1254 - val_acc: 0.9772
Epoch 759/1000
 - 8s - loss: 0.0195 - acc: 0.9959 - val_loss: 0.1409 - val_acc: 0.9743
Epoch 760/1000
 - 8s - loss: 0.0183 - acc: 0.9963 - val_loss: 0.1083 - val_acc: 0.9816
Epoch 761/1000
 - 8s - loss: 0.0191 - acc: 0.9962 - val_loss: 0.1046 - val_acc: 0.9819
Epoch 762/1000
 - 8s - loss: 0.0183 - acc: 0.9962 - val_loss: 0.1000 - val_acc: 0.9816
Epoch 763/1000
 - 8s - loss: 0.0179 - acc: 0.9962 - val_loss: 0.0990 - val_acc: 0.9823
Epoch 764/1000
 - 8s - loss: 0.0195 - acc: 0.9959 - val_loss: 0.1168 - val_acc: 0.9782
Epoch 765/1000
 - 8s - loss: 0.0209 - acc: 0.9956 - val_loss: 0.0981 - val_acc: 0.9827
Epoch 766/1000
 - 8s - loss: 0.0210 - acc: 0.9956 - val_loss: 0.1105 - val_acc: 0.9795
Epoch 767/1000
 - 8s - loss: 0.0172 - acc: 0.9967 - val_loss: 0.1107 - val_acc: 0.9802
Epoch 768/1000
 - 7s - loss: 0.0176 - acc: 0.9965 - val_loss: 0.0967 - val_acc: 0.9827
Epoch 769/1000
 - 7s - loss: 0.0199 - acc: 

 - 8s - loss: 0.0175 - acc: 0.9966 - val_loss: 0.1090 - val_acc: 0.9817
Epoch 853/1000
 - 7s - loss: 0.0183 - acc: 0.9963 - val_loss: 0.1490 - val_acc: 0.9752
Epoch 854/1000
 - 7s - loss: 0.0182 - acc: 0.9964 - val_loss: 0.1159 - val_acc: 0.9795
Epoch 855/1000
 - 7s - loss: 0.0191 - acc: 0.9962 - val_loss: 0.1102 - val_acc: 0.9803
Epoch 856/1000
 - 7s - loss: 0.0180 - acc: 0.9965 - val_loss: 0.1022 - val_acc: 0.9818
Epoch 857/1000
 - 7s - loss: 0.0172 - acc: 0.9967 - val_loss: 0.1052 - val_acc: 0.9819
Epoch 858/1000
 - 7s - loss: 0.0186 - acc: 0.9963 - val_loss: 0.1118 - val_acc: 0.9810
Epoch 859/1000
 - 7s - loss: 0.0183 - acc: 0.9963 - val_loss: 0.1015 - val_acc: 0.9816
Epoch 860/1000
 - 8s - loss: 0.0158 - acc: 0.9971 - val_loss: 0.1215 - val_acc: 0.9792
Epoch 861/1000
 - 7s - loss: 0.0188 - acc: 0.9964 - val_loss: 0.1193 - val_acc: 0.9797
Epoch 862/1000
 - 7s - loss: 0.0175 - acc: 0.9967 - val_loss: 0.1148 - val_acc: 0.9802
Epoch 863/1000
 - 7s - loss: 0.0192 - acc: 0.9963 - val_lo

Epoch 947/1000
 - 8s - loss: 0.0178 - acc: 0.9965 - val_loss: 0.1237 - val_acc: 0.9788
Epoch 948/1000
 - 8s - loss: 0.0168 - acc: 0.9967 - val_loss: 0.0971 - val_acc: 0.9833
Epoch 949/1000
 - 8s - loss: 0.0190 - acc: 0.9962 - val_loss: 0.1062 - val_acc: 0.9825
Epoch 950/1000
 - 8s - loss: 0.0188 - acc: 0.9963 - val_loss: 0.1268 - val_acc: 0.9782
Epoch 951/1000
 - 8s - loss: 0.0206 - acc: 0.9957 - val_loss: 0.1108 - val_acc: 0.9817
Epoch 952/1000
 - 8s - loss: 0.0201 - acc: 0.9960 - val_loss: 0.1261 - val_acc: 0.9788
Epoch 953/1000
 - 8s - loss: 0.0177 - acc: 0.9967 - val_loss: 0.1219 - val_acc: 0.9791
Epoch 954/1000
 - 8s - loss: 0.0161 - acc: 0.9970 - val_loss: 0.1245 - val_acc: 0.9793
Epoch 955/1000
 - 8s - loss: 0.0184 - acc: 0.9964 - val_loss: 0.1152 - val_acc: 0.9803
Epoch 956/1000
 - 8s - loss: 0.0181 - acc: 0.9964 - val_loss: 0.1151 - val_acc: 0.9806
Epoch 957/1000
 - 8s - loss: 0.0172 - acc: 0.9967 - val_loss: 0.1322 - val_acc: 0.9789
Epoch 958/1000
 - 8s - loss: 0.0177 - acc: 

In [13]:
preds = model.predict(test_data)

predictions = [i.argmax() for i in preds]
y_true = [i.argmax() for i in test_labels]
cm = confusion_matrix(y_pred=predictions, y_true=y_true)

print('Accuracy {}'.format(accuracy_score(y_true=y_true, y_pred=predictions)))

Accuracy 0.9355575195168655


In [14]:
total=sum(sum(cm))

sensitivity = cm[0,0]/(cm[0,0]+cm[1,0])
print('Sensitivity : ', sensitivity*100 )

Specificity = cm[1,1]/(cm[1,1]+cm[0,1])
print('Specificity : ', Specificity*100 )

Sensitivity :  100.0
Specificity :  98.95615866388309
